# medlineplus

In [30]:
from bs4 import BeautifulSoup
import requests
import string
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader

# url = 'https://medlineplus.gov/druginfo/drug_Aa.html'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

#Aa, Bb...
alphabets = [f'{upper}a' for upper in string.ascii_uppercase]

urls = []
for alphabet in alphabets:
    parent_url=f'https://medlineplus.gov/druginfo/drug_{alphabet}.html'
    res = requests.get(parent_url, headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    hrefs = ['https://medlineplus.gov/druginfo'+a['href'][1:] for a in soup.find('ul', id='index').find_all('a') if 'href' in a.attrs]
    urls += hrefs
# 중복제거
urls=list(set(urls))

tokenizer = tiktoken.get_encoding('cl100k_base')
def token_length(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=token_length
)

all_info = []
for url in urls:
    docs = WebBaseLoader(url).load_and_split(text_splitter)
    all_info += docs

len(all_info)
# 30m

USER_AGENT environment variable not set, consider setting it to identify your requests.


21704

# json file로 저장

In [55]:
import json

medlineplus_drug_info = []
for info in all_info:
    medlineplus_drug_info.append(
        {
            'page_content':info.page_content,
            'metadata':info.metadata
         }
    )

with open('medlineplus_drug_info.json', 'w') as json_file:
    json.dump(medlineplus_drug_info, json_file, indent=4)